<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ocp22_load_pretrained_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use OCP22 pretrained models for JARVIS-Leaderboard

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os
if not os.path.exists('ocp'):
     !git clone https://github.com/Open-Catalyst-Project/ocp.git
os.chdir('ocp')
!pip install -e .

Cloning into 'ocp'...
remote: Enumerating objects: 11184, done.
remote: Counting objects: 100% (3409/3409), done.
remote: Compressing objects: 100% (1399/1399), done.
remote: Total 11184 (delta 2332), reused 2914 (delta 1973), pack-reused 7775
Receiving objects: 100% (11184/11184), 103.58 MiB | 15.15 MiB/s, done.
Resolving deltas: 100% (7588/7588), done.
Obtaining file:///content/ocp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ocp-models (pyproject.toml) ... done
  Created wheel for ocp-models: filename=ocp_models-0.0.3-0.editable-py3-none-any.whl size=3457 sha256=f45c833511bc798d12a7cd621dab18cf82e44d3462eb7a1fb7acb5b62e90597b
  Stored in directory: /tmp/pip-ephem-wheel-cache-m_gdh88g/wheels/29/db/87/61b8ac706262b22238c01b070b675db289c051f2d890c74f3b
Successfully built ocp-models


In [3]:
pip -q install  lmdb pymatgen==2023.5.10 ase==3.22.1 orjson e3nn==0.2.6 wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.5/367.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os, torch
from ase.io import read
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import SchNet,GemNetOC,GemNetT

In [9]:
# !wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_all_s2ef.pt
# !wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gndt_oc22_all_s2ef.pt
!wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_05/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt
# !wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_finetune_all_s2ef.pt


--2023-08-30 04:18:51--  https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_finetune_all_s2ef.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.162.120, 108.157.162.108, 108.157.162.35, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.162.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155755499 (149M) [binary/octet-stream]
Saving to: ‘gnoc_finetune_all_s2ef.pt’

gnoc_finetune_all_s 100%[===================>] 148.54M  18.3MB/s    in 7.9s    

2023-08-30 04:19:00 (18.9 MB/s) - ‘gnoc_finetune_all_s2ef.pt’ saved [155755499/155755499]



In [11]:
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
fname = "gnoc_oc22_oc20_all_s2ef.pt" #"gnoc_oc22_all_s2ef.pt"
attrs = torch.load(fname, map_location=device)["config"][
    "model_attributes"
]
weights_path = fname
model = GemNetOC(None, None, 1, **attrs)

ckpt = torch.load(weights_path,map_location=torch.device('cpu'))

state_dict = ckpt["state_dict"]
state_dict = {k[2 * len("module.") :]: v for k, v in state_dict.items()}

model.load_state_dict(state_dict, strict=True)
model.to(device)

GemNetOC(
  (radial_basis): RadialBasis(
    (envelope): PolynomialEnvelope()
    (rbf): GaussianBasis()
  )
  (cbf_basis_qint): CircularBasisLayer(
    (radial_basis): RadialBasis(
      (envelope): PolynomialEnvelope()
      (rbf): GaussianBasis()
    )
  )
  (sbf_basis_qint): SphericalBasisLayer(
    (radial_basis): RadialBasis(
      (envelope): PolynomialEnvelope()
      (rbf): GaussianBasis()
    )
  )
  (radial_basis_aeaint): RadialBasis(
    (envelope): PolynomialEnvelope()
    (rbf): GaussianBasis()
  )
  (cbf_basis_aeint): CircularBasisLayer(
    (radial_basis): RadialBasis(
      (envelope): PolynomialEnvelope()
      (rbf): GaussianBasis()
    )
  )
  (cbf_basis_eaint): CircularBasisLayer(
    (radial_basis): RadialBasis(
      (envelope): PolynomialEnvelope()
      (rbf): GaussianBasis()
    )
  )
  (radial_basis_aint): RadialBasis(
    (envelope): PolynomialEnvelope()
    (rbf): GaussianBasis()
  )
  (cbf_basis_tint): CircularBasisLayer(
    (radial_basis): RadialBasis(
 